# Tutorial 4: Multi-level column indices (`MultiIndex`)

A MultiIndex is an index with multiple, hierarchical levels. We use a multiindex for the column headers of a 
dataframe. This allows each column to have multiple keys associated with it--one key for each level. 

The multiindex is a native part of the `pandas` package. For more documentation, see their [MultiIndex / advanced indexing page](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

MultiIndexes make it easy to associate multiple identifiers with a single column.  To show what this looks like we'll look at phosphoproteomics data in the endometrial cancer dataset.

In [1]:
import cptac
en = cptac.Ucec()
en.list_data_sources()

,Data type,Available sources
0,CNV,washu
1,circular_RNA,bcm
2,clinical,mssm
3,deconvolution_cibersort,washu
4,deconvolution_xcell,washu
5,follow-up,mssm
6,medical_history,mssm
7,miRNA,washu
8,phosphoproteomics,umich
9,proteomics,umich


In [ ]:
en.get_proteomics('umich')
en.get_phosphoproteomics('umich')
en.get_CNV('washu')
#NOTE: Some data may take multiple minutes to load

# Join functions with multiindices 
The join functions have been written to handle multiindices. More information on the join functions can be found in the joining_dataframes tutorial. 
An example of joining a multiindexed dataframe (in this case phosphoproteomics) with a non multiindexed dataframe (in this case CNV) is below. 

In [ ]:
phospho_and_CNV = en.join_omics_to_omics(df1_name="CNV", df2_name="phosphoproteomics", df1_source='washu', df2_source = 'umich')
phospho_and_CNV.head()

Since the CNV dataframe doesn't have the level "Site", it is filled in with NANs, so that it can be joined to the phosphoproteomics dataframe. 

# How to select from multiindex


## Selecting based on all levels
We can select single columns by passing the proper keys for all levels of the multiindex. For example, to get the proteomics for ARF5, we'd do the following:

In [ ]:
prot = en.get_proteomics('umich')
all_levels_selection = prot["ARF5"]
all_levels_selection.head(10)

## Selecting based on one level
We can easily select multiple columns from our multiindex dataframe, based on just the "Name" level of the multiindex:

In [ ]:
gene1_filter = prot.columns.get_level_values("Name").str.startswith("A") # Select all columns where the gene starts with "A". This will grab every column where the key "Name" starts with AA
gene1_data = prot.loc[:, gene1_filter]
gene1_data.head()

## Selecting based on a different level of the multiindex
We can also select based on one of the inner levels of the multiindex. For example, to get data for all tyrosine phosphorylation sites:

In [ ]:
y_site_filter = prot.columns.get_level_values("Database_ID").str.contains("ENSp") # Create a boolean filter selecting all columns where the Site level contains a "Y"

y_sites = prot.loc[:, y_site_filter] # Select the columns
y_sites.head()

# How to use `cptac.utils.reduce_multiindex()`
To make it easier to work with multi-level indices, we provide the `reduce_multiindex` function, available for import from the `cptac.utils` submodule. It can both drop levels from a multiindex, and "flatten" a multi-level index into a single-level index by concatenating the keys from multiple levels into a single key for each column.

In [ ]:
import cptac.utils as ut

## Dropping Levels
We can drop levels based on index or name. We can also drop single or multiple levels at once. 
Note that it will warn you if duplicate column key combinations arise due to dropping levels. 

### Dropping by index or name

In [ ]:
ut.reduce_multiindex(df=prot, levels_to_drop="Database_ID").head()

### Dropping single or multiple levels at once
By passing a list (or array-like) to levels_to drop, we can drop multiple levels of the multiindex at the same time. Note that we must leave at least one existing level. 

We will show this with the colon data.

In [ ]:
#colon = cptac.Colon()
#phospho = colon.get_proteomics()
#phospho.head()
cptac.get_cancer_options()

In [ ]:
# Drop evey level except Database_ID
drop = ["Name", "Site"]
ut.reduce_multiindex(df=phospho, levels_to_drop=drop).head()

## Combining levels (Flattening)

We can combine levels of a multiindexed dataframe. When combined the levels will be sepereated by an underscore, by default. We could specify a different seperator using the `sep` parameter.

In [ ]:
ut.reduce_multiindex(df=phospho, flatten=True).head()

When flatteing levels , NaNs and empty strings will automitically be dropped.

In [ ]:
phospho_and_CNV = en.join_omics_to_omics(df1_name="CNV", df2_name="phosphoproteomics")
phospho_and_CNV.head()

# Note that the CNV columns all have empty strings in the "Site" level of the columns,
# since the CNV data doesn't have any values for that.

In [ ]:
ut.reduce_multiindex(df=phospho_and_CNV, flatten=True).head()
# Notice that the empty strings have been dropped

## Getting a single level index of tuples

You can also use `reduce_multiindex` to turn the multi-level column index into a single level index of tuples, with each value in a column's tuple corresponding to the column's value for that level of the index:

In [ ]:
ut.reduce_multiindex(df=phospho, tuples=True).head()

## Turning off warnings

If your multiindex operation creates duplicate column headers, or has no effect, `reduce_multiindex` will warn you. You can silence these warnings by passing `True` to the `quiet` parameter:

In [ ]:
ut.reduce_multiindex(df=phospho, levels_to_drop=["Name", "Database_ID"]).head()

In [ ]:
# No warning will be issued
ut.reduce_multiindex(df=phospho, levels_to_drop=["Name", "Database_ID"], quiet=True).head()